In [20]:
import requests
import praw
import datetime
from dateutil.relativedelta import relativedelta
import concurrent.futures
import pandas as pd

## Data Collection

In [21]:
reddit = praw.Reddit("bot1")

In [32]:
def get_post_data_for_range(start_date, end_date):
    query = f'https://api.pushshift.io/reddit/search/submission/?subreddit=gadgets&size=500&before={end_date}&after={start_date}'
    post_data = requests.get(query)
    return post_data.json()['data']
    
def filter_submission(total_submission_dict, post):
    submission = reddit.submission(id=post['id'])
    if (submission.link_flair_text is not None):
        total_submission_dict['date'].append(post['created_utc'])
        total_submission_dict['id'].append(post['id'])
        total_submission_dict['flair'].append(submission.link_flair_text)
        total_submission_dict['title'].append(post['title'])
        total_submission_dict['selftext'].append(post['selftext'])

def get_post_data(start_date, end_date):
    total_submission_dict = {
        'date': [], 
        'id': [],
        'flair': [],
        'title': [],
        'selftext': []
    }
    
    post_data = get_post_data_for_range(start_date, end_date)
    for x in range (5):
        for submission in post_data:
            filter_submission(total_submission_dict, submission)

        start_date = post_data[-1]['created_utc']
        post_data = get_post_data_for_range(start_date, end_date)
    return total_submission_dict
    

In [41]:
#Not functional due to 
start_date = ((datetime.datetime.utcnow() - relativedelta(years=1)))
dates = [int((start_date + relativedelta(months=x)).timestamp()) for x in range(12)]
with concurrent.futures.ThreadPoolExecutor() as executor:
    threads= [executor.submit(get_post_data, dates[month], dates[month+1]) for month in range(0,len(dates),2)]
    post_data_per_month = [thread.result() for thread in threads]

In [33]:
start_date = ((datetime.datetime.utcnow() - relativedelta(days=15))).timestamp()
end_date = datetime.datetime.utcnow().timestamp()
last_15_days_data = get_post_data(1604881881, 1606100329)

In [43]:
df = pd.DataFrame(last_15_days_data)
df.to_csv('gadges_posts.csv')
df['flair'].value_counts()

VR / AR                 97
Phones                  30
Desktops / Laptops      23
Computer peripherals    14
Transportation          11
Wearables               11
Medical                 11
Home                    10
Gaming                  10
Discussion               9
Music                    7
Watches                  6
Rule 2                   5
TV / Projectors          5
Misc                     4
Cameras                  4
Homemade                 3
Phone Accessories        2
Not A Gadget             2
Blogspam                 1
Drones / UAVs            1
Tablets                  1
locked                   1
*SMS-based               1
Rule 5                   1
Name: flair, dtype: int64